# <center>پیدا کردن ربات</center>

<div dir="rtl" style="font-size:18px">
احتمالا کارتون WALL-E را بخاطر دارید. رباتی که توانست زمین را نجات دهد و باقیمانده نسل انسان‌ها را به زمین برگرداند. پس از بازگشت انسان‌ها به زمین و پس از محبوبیت زیاد WALL-E، او دوستان زیادی پیدا کرد. بهترین دوست او WALL-F یکی از بازیگوش‌ترین ربات‌هایی است که تاحالا دیده‌اید.
<br>
یکی از دفعاتی که WALL-F به بیرون رفته بود، راهی به درون یک هزار‌تو میابد. تصویر زیر از او در این هزارتو گرفته شده است.
</div>
<br>
<br>
<center>

![WALL-F Ai-Generated](Assets/robot.jpg)

</center>

<div dir="rtl">
در ادامه میخواهیم با روش‌های مختلفی پیدا کنیم که در حال حاضر ربات در چه خانه‌ای قرار دارد. در هرکدام از تست‌کیس‌هایی که لازم است بررسی کنید. خانه 0 و 0 خانه بالا و چپ هزارتو خواهد بود. در ادامه برای هر خانه مشخص خواهد شد که دیواری به هرکدام از جهات دارد یا خیر. 
به طور مثال عبارت
''R L''
به این معنی است که این خانه فقط به چپ و راست خود راه دارد. همچنین شماره‌گذاری خانه‌ها به صورت زیر است.
در هر مرحله او به شما میگوید که در کدام خانه از جدول است. عددی که ربات به شما میگوید یک عدد تصادفی است که در یک ماتریس به شما داده میشود. عدد سطر 
$i$ 
و ستون 
$j$ 
احتمال این را نشان می‌دهد که ربات در خانه 
$i$ 
باشد اما خانه 
$j$ 
را شما گزارش دهد. این ربات در پایان هر مرحله با احتمال برابر به یکی از خانه‌های مجاور میرود. همینطور خانه شروع این ربات هم توزیعی یکنواخت دارد.
</div>
<br>
<br>
<center>

![grid](Assets/grid.jpg)

</center>

<div dir="rtl">
در ابتدا برای پیاده‌سازی ۲ الگوریتمی که در ادامه خواهید دید به آبجکتی از نوع Grid نیاز خواهید داشت. عملکرد این آبجکت را میتوانید از مسیر Assets/helpers.py ببینید. به طور مثال هزارتو زیر را در نظر بگیرید.
</div>

<center>

![maze](Assets/grid2.jpg)

</center>

<div dir="rtl">
grid متناظر با این هزارتو را میتوانید از طریق تابع get_sample_grid بگیرید و با توابع آن آشنا شوید. در صورت لزوم میتوانید هر تابع دلخواهی به این کلاس اضافه کنید. برای اطلاع از تعداد سطرهای grid میتوانید از grid.n و برای اطلاع از تعداد ستون‌ها میتوانید از grid.m استفاده کنید.
</div>

In [2]:
from Assets.helpers import get_sample_grid

In [5]:
grid = get_sample_grid()

# Particle Filtering

<div dir="rtl">
در این قسمت باید آخرین خانه‌ای که این ربات در آن قرار دارد را با استفاده از الگوریتم Particle Filtering پیدا کنید. برای اینکار در ابتدا تعداد Particle هایی با تعدادی از اردر تعداد خانه‌های هزارتو بسازید. در پایان هر مرحله جمعیت را با توجه به وزن آنها نصف کنید و ذراتی جدید متناسب با تعداد ذرات درون هرخانه بسازید. در پایان خانه‌ای که بیشترین ذرات در آن قرار دارد را برگردانید.
</div>

In [13]:
# todo delete cell
import numpy as np
from Assets.helpers import Grid

In [16]:
def PF(grid: Grid, prob, obs) -> int:
    """
    input:

    grid: object of type Grid
    prob: matrix of observation probabilities
    obs: array of ints, each one is an observation

    output: 
    number of the last cell
    """
    num_cells = grid.n * grid.m

    # create transition matrix
    transition_mat = np.zeros((num_cells, num_cells))
    for i in range(num_cells):
        for j in grid.get_neighbors(i):
            transition_mat[i, j] = 1
        transition_mat[i] = transition_mat[i] / np.sum(transition_mat[i])
    
    all_cells = [i for i in range(num_cells)]
    samples = []
    num_samples = 1000
    weights = [0] * num_samples
    cell_weight = [0] * num_cells

    # init samples
    samples = np.random.randint(num_cells, size=num_samples)

    for observed_index, observed in enumerate(obs):
        # calculate weights, calculate weight for each cell
        cell_weight = [0] * num_cells
        for idx, sample in enumerate(samples):
            weights[idx] = prob[sample, observed]
            cell_weight[sample] += weights[idx]
        cell_weight = np.array(cell_weight) / np.sum(cell_weight)

        samples = np.random.choice(all_cells, p=cell_weight, size=num_samples)

        if observed_index + 1 == len(obs):
            return np.argmax(cell_weight)
        else:
            for i in range(num_samples):
                samples[i] = np.random.choice(all_cells, p=transition_mat[samples[i]])

In [17]:
from Assets.helpers import PF_checker
PF_checker(PF)

73
PF PASSED!


# HMM

<div dir="rtl">
در این قسمت غیر از اینکه باید بفهمیم WALL-F در کدام خانه قرار دارد. میخواهیم بدانیم که محتمل‌ترین مسیری که طی کرده‌ است چه مسیری است. برای اینکار الگوریتم viterbi را پیاده‌سازی کنید.
</div>

In [1]:
# todo delete this cell
import numpy as np
import random
from Assets.helpers import Grid

In [11]:
from Assets.helpers import consistent_log

def viterbi(grid, prob, obs):
    """
    input:

    grid: object of type Grid
    prob: matrix of observation probabilities
    obs: array of ints, each one is an observation

    output: 
    array of cell numbers, same shape as obs
    """
    num_cells = grid.m * grid.n
    mat_prob, mat_move = np.zeros((num_cells, len(obs))), -np.ones((num_cells, len(obs)), dtype=np.int32)

    # create transition matrix
    transition_mat = np.zeros((num_cells, num_cells))
    for i in range(num_cells):
        for j in grid.get_neighbors(i):
            transition_mat[i, j] = 1
        transition_mat[i] = transition_mat[i] / np.sum(transition_mat[i])

    # init first state probabilities
    for i in range(num_cells):
        mat_prob[i, 0] = consistent_log(prob[i, obs[0]])
    
    # DP
    for t in range(1, len(obs)):
        for i in range(num_cells):
            arr_temp = np.array([consistent_log(j) for j in transition_mat[:, i]]) + consistent_log(prob[i, obs[t]])
            mat_move[i, t] = np.argmax(mat_prob[:, t - 1] + arr_temp)
            mat_prob[i, t] = (mat_prob[:, t - 1] + arr_temp)[mat_move[i, t]]
    
    arr_ret = [np.argmax(mat_prob[:, -1])]
    t = len(obs) - 1

    while t > 0:
        arr_ret.append(mat_move[arr_ret[-1], t])
        t -= 1
    
    return arr_ret[::-1]



In [12]:
from Assets.helpers import viterbi_checker
viterbi_checker(viterbi)

VITERBI PASSED!
